In [1]:
# -*- coding: utf-8 -*-
import os
import sys
import pandas as pd
import xml.etree.ElementTree as ET

TAG_STRING_TABLE_ENTRY = "Entry"
TAG_STRING_TABLE_NAME = 'Name'
TAG_STRING_TABLE_ID = 'ID'
TAG_STRING_TABLE_DEFAULT_TEXT = "DefaultText"
TAG_STRING_TABLE_FEMALE_TEXT = "FemaleText"


class StringTable:
    def __init__(self):  
        self.df = pd.DataFrame()

    def parse_file(self, filename):
        # parse string table xml
        tree = ET.parse(filename)
        root = tree.getroot()

        print(filename)
        name = root.find(TAG_STRING_TABLE_NAME).text
        items = []
        for entry in root.iter(TAG_STRING_TABLE_ENTRY):
            id = entry.find(TAG_STRING_TABLE_ID).text
            default_text = entry.find(TAG_STRING_TABLE_DEFAULT_TEXT).text
            female_text = entry.find(TAG_STRING_TABLE_FEMALE_TEXT).text
            
            item = (name, id, default_text, female_text)
            items.append(item)
        return items

    def parse_dir(self, strtbl_dir):
        result = []
        for dirpath, dirnames, filenames in os.walk(strtbl_dir):
            for filename in filenames:
                if filename.startswith(("~", ".")):
                    continue
                if not filename.endswith(("stringtable")):
                    continue
                full_filename = os.path.join(dirpath, filename)
                items = self.parse_file(full_filename)
                result.extend(items)
        return result

        
    def create_df(self, **args):
        '''
        Args:
            index {boolean}
            data {list}
            d_key {str}
            f_key {str}
        '''
        index = True  if not 'index' in args else args['index']
        d_key = 'DefaultText' if not 'd_key' in args else args['d_key']
        f_key = 'FemaleText' if not 'f_key' in args else args['f_key']
        
        if not 'data' in args:
            print('err: need Data')
            return
            
        df = pd.DataFrame.from_records(data, columns=['Name', 'ID', d_key, f_key])
        if index:
            df.index = [ df['Name'] +'_'+ df['ID'] ]
            df.index.name = 'index'

        return df
            
            
odd = StringTable()
# filename = os.path.join(sys.path[0], 'file\gui.stringtable') 
# odd.parse_file(filename)
# odd.parse_dir('file')
# odd.create_df(data=odd.parse_dir('file'), d_key = 'm_d')


In [10]:
data = [('game\\gui', '0', 'Godandag', None),
 ('game\\gui', '1', 'Cönyngsdag', None),
 ('game\\gui', '2', 'Mecwynsdag', None)]

# woc = pd.DataFrame(data, columns=['Name', 'ID', 'DefaultText', 'FemaleText'])
# pd.DataFrame.from_records(data,columns=['Name', 'ID', 'DefaultText', 'FemaleText'])
odd.create_df(data=data)

,Name,ID,DefaultText,FemaleText
index,,,,
game\gui_0,game\gui,0,Godandag,None
game\gui_1,game\gui,1,Cönyngsdag,None
game\gui_2,game\gui,2,Mecwynsdag,None
